In [1]:
!pip install scikit-optimize

In [2]:
import keras
import tensorflow
import skopt
import pandas as pd

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops
from skopt import gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer

Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train/ 255
X_test = X_test/ 255
X_train = X_train.reshape(60000,784)
X_test = X_test.reshape(10000,784)
input_shape= X_train[0].shape

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [0]:
dim_learning_rate = Real(low=1e-3, high=1e-1, prior='log-uniform', name='learning_rate')
dim_num_dense_layers = Integer(low=0, high=2, name='num_dense_layers')
dim_num_dense_nodes = Integer(low=5, high=15, name='num_dense_nodes')
dim_optimizer = Categorical(categories=['Adam', 'SGD'], name='optimizer')

dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_dense_nodes,
              dim_optimizer]
default_parameters = [1e-3, 2, 13, 'SGD']

In [0]:
def create_model(learning_rate, num_dense_layers, num_dense_nodes, optimizer):
    model = Sequential()
    model.add(Dense(num_dense_nodes, input_shape= input_shape, activation='relu'))
    for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)
        model.add(Dense(num_dense_nodes, activation='relu', name=name))
    model.add(Dense(10,activation='softmax'))
    if optimizer == 'Adam':
      optimizer_final = Adam(lr=learning_rate)
    if optimizer == 'SGD':
      optimizer_final = SGD(lr=learning_rate)
    model.compile(optimizer=optimizer_final, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [0]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_dense_nodes, optimizer):

    model = create_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         optimizer=optimizer)    

    blackbox = model.fit(x=X_train, y=y_train, epochs=10, batch_size=32, validation_split=0.15,)
    accuracy = blackbox.history['val_accuracy'][-1]
    print("Accuracy: {0:.2%}".format(accuracy))

    del model
    K.clear_session()
    ops.reset_default_graph()    

    return -accuracy

In [0]:
K.clear_session()
ops.reset_default_graph()

In [8]:
gp_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=12,
                            n_jobs=-1,
                            kappa = 5,
                            x0=default_parameters)

Train on 51000 samples, validate on 9000 samples
Epoch 1/10
51000/51000 [==============================] - 3s 52us/step - loss: 2.2200 - accuracy: 0.1819 - val_loss: 2.1139 - val_accuracy: 0.2710
Epoch 2/10
51000/51000 [==============================] - 2s 49us/step - loss: 1.9757 - accuracy: 0.3238 - val_loss: 1.8179 - val_accuracy: 0.3941
Epoch 3/10
51000/51000 [==============================] - 2s 48us/step - loss: 1.6850 - accuracy: 0.4710 - val_loss: 1.4976 - val_accuracy: 0.5654
Epoch 4/10
51000/51000 [==============================] - 3s 50us/step - loss: 1.3813 - accuracy: 0.5683 - val_loss: 1.1933 - val_accuracy: 0.5973
Epoch 5/10
51000/51000 [==============================] - 2s 49us/step - loss: 1.1374 - accuracy: 0.6053 - val_loss: 0.9948 - val_accuracy: 0.6546
Epoch 6/10
51000/51000 [==============================] - 2s 49us/step - loss: 0.9824 - accuracy: 0.6732 - val_loss: 0.8692 - val_accuracy: 0.7347
Epoch 7/10
51000/51000 [==============================] - 3s 49us/ste

In [9]:
print("best accuracy was " + str(round(gp_result.fun *-100,2))+"%.")
gp_result.x

best accuracy was 95.42%.


[0.06883953783942746, 1, 14, 'SGD']

In [12]:
gp_model = create_model(gp_result.x[0],gp_result.x[1],gp_result.x[2],gp_result.x[3])
gp_model.fit(X_train,y_train, batch_size=10, epochs=10)
gp_model.evaluate(X_test,y_test)

Epoch 1/10
60000/60000 [==============================] - 7s 123us/step - loss: 0.3828 - accuracy: 0.8824
Epoch 2/10
60000/60000 [==============================] - 7s 121us/step - loss: 0.2532 - accuracy: 0.9247
Epoch 3/10
60000/60000 [==============================] - 7s 122us/step - loss: 0.2235 - accuracy: 0.9333
Epoch 4/10
60000/60000 [==============================] - 7s 120us/step - loss: 0.2047 - accuracy: 0.9403
Epoch 5/10
60000/60000 [==============================] - 7s 121us/step - loss: 0.1926 - accuracy: 0.9436
Epoch 6/10
60000/60000 [==============================] - 7s 122us/step - loss: 0.1828 - accuracy: 0.9460
Epoch 7/10
60000/60000 [==============================] - 7s 122us/step - loss: 0.1744 - accuracy: 0.9490
Epoch 8/10
60000/60000 [==============================] - 7s 124us/step - loss: 0.1722 - accuracy: 0.9492
Epoch 9/10
60000/60000 [==============================] - 7s 121us/step - loss: 0.1667 - accuracy: 0.9512
Epoch 10/10
10000/10000 [=====================

[0.18893614380545914, 0.9449999928474426]

In [13]:
gp_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 14)                10990     
_________________________________________________________________
layer_dense_1 (Dense)        (None, 14)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                150       
Total params: 11,350
Trainable params: 11,350
Non-trainable params: 0
_________________________________________________________________
